In [ ]:
import importlib
from pprint import pprint

from cuery import search
from cuery.seo import geo
from cuery.utils import set_env

importlib.reload(search)

set_env(apify_secrets=False)

In [ ]:
resp = await search.query_deepseek(
    prompt="Mejores SUVs en España 2025 precio-calidad",
    model="deepseek-chat",
    use_search=True,
    validate=True,
)

print(resp)
print(resp.references)

In [ ]:
int(6 / 20)

# LLM prompting with web search

In [ ]:
# model = "openai/gpt-4.1-mini"
model = "google/gemini-2.5-flash"
# model = "xai/grok-3"
# model = "deepseek/deepseek-chat"

prompts = [
    "Mejores SUVs en España 2025 precio-calidad",
    "Cuáles son los mejores SUVs chinos dispnibles en España 2025",
]

responses = await search.query(prompts, model=model, max_concurrent=10, use_search=True)
responses.to_pandas()

# Competitor extraction

In [ ]:
importlib.reload(geo)

competitors = await geo.find_competitors(
    brands=["BYD"],
    sector="automotive",
    market="Spain",
    max_count=5,
)
competitors

In [ ]:
competitors = ["toyota", "kia", "mg", "renault", "omoda"]

In [ ]:
from cuery.seo import geo

importlib.reload(geo)

prompts = await geo.commercial_prompts(
    n=10,
    language="en",
    sector=None,  # "automotive",
    market=None,  # "Spain",
    brands=["BYD"] + competitors,
    include_brands="never",
)

pprint(prompts)

In [ ]:
importlib.reload(search)
importlib.reload(geo)

use_search = False
models = ["openai/gpt-4.1", "google/gemini-2.5-flash"]
# models = ["deepseek/deepseek-chat"]

df = await geo.query_with_models(prompts, models, use_search=use_search)
df

In [ ]:
df.to_parquet("geo_results.parquet", index=False)

In [ ]:
df = df[[col for col in df if not any(x in col for x in ("_ranks", "_pos"))]].copy()
df

In [ ]:
importlib.reload(geo)

df2 = geo.add_brand_ranks(
    df,
    brands=["Opel", "Dacia", "BYD", "Kia", "Renault", "MG", "Omoda", "Jaecoo"],
)
df2

# All-in-one

In [ ]:
importlib.reload(geo)


async def callback(progress: dict):
    n = progress.get("n", 0)
    total = progress.get("total", "?")
    print(f"Processed: {n}/{total} rows.")


res = await geo.query_with_models(
    prompts=["best electric SUVs for families in 2024", "best electric SUVs under $50,000"],
    models=["openai/gpt-4.1", "google/gemini-2.5-flash"],
    use_search=True,
    progress_callback=callback,
)

In [ ]:
importlib.reload(geo)

params = {
    "models": ["openai/gpt-4.1", "google/gemini-2.5-flash", "deepseek/deepseek-chat"],
    "prompts_seed": ["best electric SUVs for families in 2024"],
    "prompts_max": 10,
    "prompts_language": "English",
    "brands_seed": ["Peugeot"],
    "brands_max": 10,
    "brands_model": "openai/gpt-4.1",
    "brands_in_prompt": "never",
    "sector": "automotive",
    "market": "Spain",
    "use_search": True,
}

cfg = geo.GeoConfig(**params)
print(cfg)

result = await geo.analyse(cfg)
result